In [ ]:
'''
This notebook should generate Supplementary Figures 3
'''

In [ ]:
import numpy as np
%matplotlib inline
import matplotlib as mpl
plt = mpl.pyplot
#from matplotlib import pyplot as plt
import seaborn as sns
from collections import namedtuple as nt
from collections import defaultdict as ddict
from copy import deepcopy
import pickle

import sys
sys.path.append("./../") # adds parent directory to path
import os
import stationary_distribution_aug as sd
import mutator_classes as mc
import common_functions as cf

In [ ]:
# plotting stuff
sns.set_style('whitegrid')

mpl.rcParams['figure.dpi'] = 400
mpl.rcParams['axes.titlesize'] = 10
mpl.rcParams['axes.labelsize'] = 10
mpl.rcParams['xtick.labelsize'] = 10
mpl.rcParams['ytick.labelsize'] = 10
mpl.rcParams['legend.fontsize'] = 10
mpl.rcParams['legend.title_fontsize'] = 10

plt = mpl.pyplot
mlines = mpl.lines

title_height = 1.1
within_color = 'r'
between_color = 'k'
mean_color = 'k'
line_weight = 2
marker_weight = 5

In [ ]:
# helper functions to calculate summaries of simualted results
def standard_error(i,n):
    return i/(n**0.5)

result_tuple = nt('result_tuple',['s','mean','mean_err','variance','variance_err','within','within_err','phi'])

def create_result_tuple_obs(l):
    
    return result_tuple(s = l[0],
                        mean=l[1],
                        mean_err = l[2],
                        variance = l[3],
                        variance_err = l[4],
                        within = l[5],
                        within_err = l[6],
                        phi = l[7])

def get_s(p):
    s = 4*p.N*p.h*p.s*p.loci*p.phi
    return s

# calculate summaries of simulated results
def process_obs(a,rdict):
    p = list(a.keys())[0]
#     print(p)
    s = get_s(p)
    m = p.M
        
    
    values = list(a.values())[0]
    mean = values[0]
    mean_err = values[1][0]
    
    variance_values = values[4]
    variance_values = variance_values[p]['var']
    ind_values = []
    sd_variance = []
    mean_variance = []
    within_variance = []
    for i in variance_values:
        ind_values.extend(i)
        sd_variance.append(np.var(i))
        within_variance.append(np.mean([j*(1-j) for j in i])*2)
        mean_variance.append(np.mean(i))
        
        
    mean = np.mean(ind_values)
    mean_err = np.std(mean_variance)
    
    variance = np.var(ind_values)
    variance_err = np.std(sd_variance)
#     print(mean_err/mean,variance_err/variance)

    
    ind_values = np.array(ind_values)
    within_values = np.mean(ind_values*(1-ind_values)*2)
    within = np.mean(within_values)
    within_err = np.std(within_variance)
    num_trials = len(variance_values)
    print(num_trials)
    
    mean_err = standard_error(mean_err,num_trials)
    variance_err = standard_error(variance_err,num_trials)
    within_err = standard_error(within_err,num_trials)
    
    results = create_result_tuple_obs([s,mean,mean_err,variance,variance_err,within,within_err,p.phi])
    rdict[m][s] = results

In [ ]:
# find files for the different alternative parameter combinations
in_dir = '/Users/will_milligan/PycharmProjects/Mutator_Project/Final Figs/scaled_sim_results/'
files = os.listdir(in_dir)
alt_param_names = set([os.path.dirname(a).split('/')[-1] for a in files])
print(alt_param_names)

# for each alternative parameter combination, load and summarize reuslts
results = {}
for i in alt_param_names:
    results[i] = ddict(dict)

params = {}    
for name in files:
    alt_name = os.path.dirname(name).split('/')[-1]
    if alt_name == 'DEFAULT' or alt_name == 'SMALLu0' or alt_name == 'LARGEu0': continue
    with open(name,'rb') as fin:
        a = pickle.load(fin)
    process_obs(a,results[alt_name])

    # store one copy of the params per alternative model
    params[alt_name] = list(a.keys())[0]
    

In [ ]:
# functions to calcualte and summarize expected results
result_tuple_exp = nt('result_tuple',['s','mean','variance','within','segregating','phi'])

def create_result_tuple_exp(l):
    
    return result_tuple_exp(s = l[0],
                        mean=l[1],
                        variance = l[2],
                        within = l[3],
                        segregating = l[4],
                        phi = l[5])

def get_phi(s,p):
    phi = s/(4*p.N*p.h*p.s*p.loci)
    return phi

def process_exp(p,s,rdict):
    p.phi = get_phi(s,p)
    statdist = sd.get_SD(p)
    frequency_mean      = sum(i*p for i,p in statdist.items())
    frequency_variance  = sum(i**2*p for i,p in statdist.items()) - frequency_mean**2
    seg_variance        = sum((i-frequency_mean)**2*p for i,p in statdist.items() if i > 0 and i < 1)
    within_variance     = sum(2*i*(1-i)*p for i,p in statdist.items())
    
    result = create_result_tuple_exp([s,frequency_mean,frequency_variance,within_variance,seg_variance,p.phi])
    rdict[s] = result

In [ ]:
# calculate expected results
exp_results = {}
for i in alt_param_names:
    if i in exp_results.keys() or i == 'DEFAULT': 
        continue
    param_value = params[i]
    exp_results[i] = {}
    for s in np.logspace(-2,3,100):
        process_exp(param_value,s,exp_results[i])


### GET THE EXPECTED VALUES ###
expected_results_sorted = {}
for name,param_results in exp_results.items():
    
    s_values_exp = np.sort(list(param_results.keys()))
    phi_exp = np.array([param_results[s].phi for s in s_values_exp])
    mean_freq_exp = np.array([param_results[s].mean for s in s_values_exp])
    var_freq_exp = np.array([param_results[s].variance for s in s_values_exp])
    within_exp = np.array([param_results[s].within for s in s_values_exp])
    seg_exp = np.array([param_results[s].segregating for s in s_values_exp])

    expected_results_sorted[name] = (s_values_exp,phi_exp,mean_freq_exp,var_freq_exp,within_exp,seg_exp)

In [ ]:
# function to turn the observed rsults into a more useful object
def get_observed_results(rdict):
    s_values = np.sort(list(rdict.keys()))
    
    phi = np.array([rdict[s].phi for s in s_values])
    
    mean_freq = np.array([rdict[s].mean for s in s_values])
    mean_err = np.array([rdict[s].mean_err for s in s_values])
    var_freq = np.array([rdict[s].variance for s in s_values])
    var_err = np.array([rdict[s].variance_err for s in s_values])
    within = np.array([rdict[s].within for s in s_values])
    within_err = np.array([rdict[s].within_err for s in s_values])
    
    return (s_values,phi,mean_freq,mean_err,var_freq,var_err,within,within_err)

In [ ]:
fig, axes = plt.subplots(figsize=(6.5,6.5/8*4.523*len(alt_param_names)),nrows=len(alt_param_names),ncols=2)
baseline = 1.25e-7/2
expected_mutation_rate = 1.25e-7

letters = list(np.flip(np.array(['A','B','C','D','E','F','G','H','I','J','K','L','M'])))
translate_names = {'LARGEmu': 'Effect of doubling mutation rate at modifier loci, $\\mu$', 
                   'SMALLu0': 'Effect of halving baseline mutation rate rate, $u_0$',
                   'LARGEu0': 'Effect of increasing baseline mutation rate rate, $u_0$ by 50\%',
                   'SMALL2Nhs': 'Effect of halving selection at selected sites rate, $Lhs$',
                   'LARGE2Nhs':'Effect of doubling selection at selected sites rate, $Lhs$'}

for index,i in enumerate(['LARGEmu', 'SMALL2Nhs', 'LARGE2Nhs']):
    ax1,ax2 = axes[index]

    translated_name = translate_names[i]
    legend_entries = {}
    M = 1
    rdict = results[i][1000]
    s_values,phi, mean_freq, mean_err, var_freq, var_err, within, within_err = get_observed_results(rdict)
    s_values_exp,phi_exp,mean_freq_exp,var_freq_exp,within_exp,seg_exp       = expected_results_sorted[i]

    
    nvals = (expected_mutation_rate-baseline)/(2*mean_freq*phi)
    nvals_exp = (expected_mutation_rate-baseline)/(2*mean_freq_exp*phi_exp)
    phi = np.array([rdict[s].phi*baseline for s in s_values])

    ax1.errorbar(s_values,
                 2*mean_freq*phi*M,
                 2*M*mean_err*phi*2,
                 color=mean_color,
                 ls='None',
                 marker='.')
    
    ax1.plot(s_values_exp,
             2*mean_freq_exp*M*phi_exp,
             color=mean_color,
             alpha=0.5,
             lw=line_weight)


        
        
    if index == 0:
        ax1.set_title('$\\bf{I.}$ Effect on mean mutation rate',y=title_height)
        ax1.text(x = 10, y = 8*baseline/1000, s = r'$\bf{A.}$ Doubled mutation rate at modifier sites, $\mu$')
    elif index == 1:
        ax1.text(x = 9, y = 17*baseline/1000, s = r'$\bf{B.}$ Doubled strength of selection at selected sites, $hs$')
    else:
        ax1.text(x = 9, y = 4.5*baseline/1000, s = r'$\bf{C.}$ Halved strength of selection at selected sites, $hs$')
#     ax1.set_title('$\\bf{I.}$ Effect on mean mutation rate',y=title_height)

    legend_entries['Analytic\napprox.']=mlines.Line2D([], [], color='k',ls='-',lw=line_weight)
    legend_entries['Simulated\nmean'+ r' $\pm$ 2 SEs']=plt.errorbar([],[],[],ls='None',marker='.',color='k',markersize = 7)
    location = 'upper right'
    plt.sca(ax1)
    
    if index == 0:
        plt.legend(np.flip(np.array(list(legend_entries.values()))),
                   np.flip(np.array(list(legend_entries.keys()))),
                   ncol=1,edgecolor='k',framealpha=1,loc=location,
                   handlelength=1,
                   fontsize=9,
                   borderaxespad=0.2,
                   borderpad=0.2)
        
    ax2.errorbar(s_values,
                    4*var_freq*phi**2*nvals,
                    4*var_err*phi**2*nvals*2,
                    color=between_color,ls='None',marker='.')
    
    ax2.plot(s_values_exp,
                    4*var_freq_exp*phi_exp**2*nvals_exp,
                    color=between_color,
                    lw=line_weight,alpha=0.5)

    ax2.errorbar(s_values,
                    within*phi**2*nvals,
                    within_err*phi**2*nvals*2,
                    color=within_color,
                    zorder=1,
                    ls='None',
                    marker='^',
                    markersize = 3)
    
    ax2.plot(s_values_exp,
         within_exp*phi_exp**2*nvals_exp,
         color=within_color,
         lw=line_weight,alpha=0.5)

    if i == 'LARGE2Nhs':
        adjustment = 0.25
    elif i == 'LARGEmu':
        adjustment  = 0.75
    else: adjustment = 1
        
    if index == 0:
        ax2.set_title('$\\bf{II.}$ Effect on variance between & within populations',y=title_height)
        ax2.text(y = 1e-6*adjustment*baseline**2,x = 0.9e-2, s = 'Within', rotation = 35,color = within_color)
        ax2.text(y = 1e-3*adjustment*baseline**2,x = 0.9e-2, s = 'Between', rotation = 30,color = between_color)
    
    ax1.set_ylabel('Increase in mean mutation rate',labelpad=0)
    ax2.set_ylabel('Variance in mutation rate',labelpad=0)

    
    for ax in [ax1,ax2]:
        ax.grid([0.9]*3)
        ax.spines['bottom'].set_color('k')
        ax.spines['top'].set_color('k')
        ax.spines['left'].set_color('k')
        ax.spines['right'].set_color('k')
        ax.set_xticks(np.logspace(-2,3,6))
        ax.set_xscale('log')
        ax.set_yscale('log')
        ax.set_xlabel('Scaled selection parameter ' + r'$\left( 4NLhs\phi\right)$')
        plt.sca(ax)
        plt.tick_params(bottom=True, top=False, left=True, right=False)
        plt.tick_params(bottom=True, top=False, left=True, right=False,which='minor')
        plt.tick_params(labelbottom=True, labeltop=False, labelleft=True, labelright=False)

plt.subplots_adjust(wspace=0.3,hspace=0.4)

ax1 = plt.gca()
bbox = ax1.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
width, height = bbox.width, bbox.height
print(round(width/height,3),width,height)